In [ ]:
from music21 import converter, note, chord, instrument
import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
class MusicMetrics():
    def __init__(self, files):
        self.files = files
        self.mean_pitch_count = None
        self.mean_chord_count = None
        self.pitch_count = None
        self.chord_count = None
        self.pitches = set()
        self.transition_matrix = None

        self.notes_to_parse = []
        for f in glob.glob(self.files):
            midi = converter.parse(f)
            parts = instrument.partitionByInstrument(midi)
            if parts:
                notes_to_parse = parts.parts[0].recurse()
            else:
                notes_to_parse = midi.flat.notes
            # temp = []
            for current_note in notes_to_parse:
                if isinstance(current_note, note.Note):
                    self.notes_to_parse.append(str(current_note.pitch))
                elif isinstance(current_note, chord.Chord):
                    self.notes_to_parse.append('.'.join(str(n) for n in current_note.normalOrder))
            # self.notes_to_parse.append(temp)

    def evaluate(self):
        note_lens = []
        chord_lens = []
        for f in glob.glob(self.files):
            notes = []
            chords = []
            notes_lst = []
            chords_lst = []
            midi = converter.parse(f)
            notes_to_parse = None
            parts = instrument.partitionByInstrument(midi)
            if parts:  # file has instrument parts
                notes_to_parse = parts.parts[0].recurse()
            else:  # file has notes in a flat structure
                notes_to_parse = midi.flat.notes
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    self.pitches.add(str(element.pitch))
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    self.pitches.add('.'.join(str(n) for n in element.normalOrder))
                    chords.append('.'.join(str(n) for n in element.normalOrder))
            notes_lst.append(len(notes))
            chords_lst.append(len(chords))
            note_lens.append(len(set(notes)))
            chord_lens.append(len(set(chords)))

        self.mean_chord_count = np.mean(chord_lens)
        self.mean_pitch_count = np.mean(note_lens)
        self.pitch_count = np.mean(notes_lst)
        self.chord_count = np.mean(chords_lst)

    def evaluate_pitch_transition_matrix(self, additional_notes):
        df = pd.DataFrame(columns=sorted(list(self.pitches.union(additional_notes))), index=sorted(list(self.pitches.union(additional_notes)))).fillna(0)
        # print(df)
        for f in glob.glob(self.files):
            midi = converter.parse(f)
            parts = instrument.partitionByInstrument(midi)
            if parts:
                notes_to_parse = parts.parts[0].recurse()
            else:
                notes_to_parse = midi.flat.notes

            previous_note = notes_to_parse[0]
            i = 1
            while not (isinstance(previous_note, note.Note) or isinstance(previous_note, chord.Chord)):
                i += 1
                previous_note = notes_to_parse[i]
            if isinstance(previous_note, note.Note):
                previous_note = str(previous_note.pitch)
            elif isinstance(previous_note, chord.Chord):
                previous_note ='.'.join(str(n) for n in previous_note.normalOrder)

            for element in notes_to_parse[i+1:]:
                # print(element, "success")
                if isinstance(element, note.Note):
                    df.at[previous_note, str(element.pitch)] += 1
                    previous_note = str(element.pitch)

                elif isinstance(element, chord.Chord):
                    df.at[previous_note, '.'.join(str(n) for n in element.normalOrder)] += 1
                    previous_note = '.'.join(str(n) for n in element.normalOrder)

                else:
                    continue
        scaler = MinMaxScaler()
        self.transition_matrix = scaler.fit_transform(df)
        return self.transition_matrix


    def evaluate_double_pitch_transition_matrix(self, tm2, sequence_len):
        seq_pitches = []
        # for parts in self.notes_to_parse:
        seq_pitches = set(tuple(self.notes_to_parse[index : index + sequence_len]) for index in range(len(self.notes_to_parse) - sequence_len))
        seq_pitches.update(tuple(tm2.notes_to_parse[index : index + sequence_len]) for index in range(len(tm2.notes_to_parse) - sequence_len))

        pitches = sorted(self.pitches.union(tm2.pitches))

        df = pd.DataFrame(columns=sorted(seq_pitches), index=pitches).fillna(0)
        previous_notes = self.notes_to_parse[:sequence_len]
        for element in self.notes_to_parse[sequence_len:]:
            df.at[str(element), tuple(previous_notes)] += 1
            previous_notes = previous_notes[1:]
            previous_notes.append(str(element))

        scaler = MinMaxScaler()
        self.transition_matrix = scaler.fit_transform(df)
        return self.transition_matrix


    def compare_pitch_transition_matrix(self, tm2):
        return np.linalg.norm(self.transition_matrix - tm2.transition_matrix)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# input path to directory with the starting data
ms_original = MusicMetrics("/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai/*.mid")
ms_original.evaluate()

In [ ]:
# input path to directory with midi files with which you want compare
files = glob.glob("/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/*.mid")

In [ ]:
print("simple")
result = []
res = []
for f in files:
  print(f)
  ms_generated = MusicMetrics(f)
  ms_generated.evaluate()
  ms_original.evaluate_pitch_transition_matrix(ms_generated.pitches)
  ms_generated.evaluate_pitch_transition_matrix(ms_original.pitches)
  value = ms_original.compare_pitch_transition_matrix(ms_generated)
  print("Distance -", value)
  res.append(value)
result.append(res)

simple
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs50.mid
Distance - 24.638776933504115
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs25.mid
Distance - 26.064596901286077
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs73.mid
Distance - 24.96522839632422
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/ty_september.mid
Distance - 26.455050938203122
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/ty_oktober.mid
Distance - 25.110793789824754
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/ty_november.mid
Distance - 25.67990878117385
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/regular_model_seq_100_output.mid
Distance - 24.093095415190614
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/regular_model_seq_100_output2.mid
Distance - 24.499320905059992
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/regular

In [ ]:
res = []
for seq_len in range(2, 10):
  print("Sequence length - ", seq_len, "\n")
  temp = []
  for f in files:
    print(f)
    ms_generated = MusicMetrics(f)
    ms_generated.evaluate()
    ms_original.evaluate_double_pitch_transition_matrix(ms_generated, seq_len)
    ms_generated.evaluate_double_pitch_transition_matrix(ms_original, seq_len)
    value = ms_original.compare_pitch_transition_matrix(ms_generated)
    print("Distance -", value)
    temp.append(value)
  res.append(temp)
result.append(res)

Sequence length -  2 

/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs50.mid


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


Distance - 60.27748634178447
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs25.mid


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


Distance - 61.64966048131807
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs73.mid


...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


Distance - 103.5867083680018
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/24445_Medley.mid


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


Distance - 103.5867083680018


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
res = []
for seq_len in range(10, 101, 10):
  print("Sequence length - ", seq_len, "\n")
  temp = []
  for f in files:
    print(f)
    ms_generated = MusicMetrics(f)
    ms_generated.evaluate()
    ms_original.evaluate_double_pitch_transition_matrix(ms_generated, seq_len)
    ms_generated.evaluate_double_pitch_transition_matrix(ms_original, seq_len)
    value = ms_original.compare_pitch_transition_matrix(ms_generated)
    print("Distance -", value)
    temp.append(value)
  res.append(temp)
result.append(res)

Sequence length -  10 

/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs50.mid


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


Distance - 80.87175032111027
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs25.mid


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


Distance - 80.96906816803562
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs73.mid


...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


Distance - 117.37973747663223
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/24445_Medley.mid


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


Distance - 117.37973747663223


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
# order of files
for elem in files:
  print(elem)

/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs50.mid
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs25.mid
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/epochs73.mid
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/ty_september.mid
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/ty_oktober.mid
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/ty_november.mid
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/regular_model_seq_100_output.mid
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/regular_model_seq_100_output2.mid
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/regular_model_seq_100_output3.mid
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/regular_model_seq_100_output4.mid
/content/drive/MyDrive/3 курс/AI/AIComposer-master/tschai_outputs/regular_model_seq_100_output5.mid
/content/drive/MyD

In [ ]:
# simple
for elem in result[0]:
  print(round(elem, 3))

24.639
26.065
24.965
26.455
25.111
25.68
24.093
24.499
26.498
23.816
24.471
26.518
28.531
28.531


In [ ]:
# seq
count = 2
for part in result[1:]:
  for lst in part:
    print("\n", count)
    count += 1
    for elem in lst:
      print(round(elem, 3))


 2
60.277
61.65
60.607
61.418
60.768
61.623
59.087
59.167
61.816
58.089
59.67
64.279
73.353
73.353

 3
71.502
72.183
71.804
72.218
71.557
72.844
70.267
70.187
72.253
68.862
70.975
75.829
89.322
89.322

 4
74.134
74.571
74.442
74.851
74.043
75.591
73.081
72.944
74.632
71.385
73.666
78.468
94.423
94.423

 5
75.745
76.104
76.053
76.566
75.653
77.319
74.847
74.674
76.163
72.992
75.348
80.113
97.26
97.26

 6
77.056
77.35
77.352
77.952
76.965
78.715
76.265
76.068
77.409
74.297
76.692
81.379
99.376
99.376

 7
78.157
78.392
78.443
79.14
78.079
79.9
77.436
77.249
78.45
75.388
77.806
82.399
101.093
101.093

 8
79.11
79.298
79.386
80.158
79.039
80.931
78.448
78.283
79.354
76.336
78.757
83.279
102.43
102.43

 9
80.034
80.176
80.3
81.138
79.963
81.93
79.43
79.287
80.232
77.254
79.677
84.132
103.587
103.587

 10
80.872
80.969
81.129
82.03
80.802
82.844
80.324
80.206
81.025
78.083
80.511
84.9
104.516
104.516

 11
86.41
86.252
86.548
87.718
86.333
88.863
86.27
86.148
86.305
83.609
86.055
89.958
110.1